In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")
pd.options.display.float_format = '{:,.2f}'.format


In [2]:
def obtener_full_path(directorio):
    return [os.path.join(directorio, file) for file in os.listdir(directorio)]


def rankear_y_plottear(df, eje_x, eje_y, titulo_grafico, numero_ranking):
    '''Esta función ordena los datos segun el eje y que se quiere plottear'''
    df_ordenada = df.sort_values(eje_y)
    diez_mas_altos = df.head(numero_ranking)
    diez_mas_bajos = df.tail(numero_ranking)

    mosaico = '''
    AB
    AC
    '''
    plot_args = [df, eje_x, eje_y, titulo_grafico]

    fig, axis = plt.subplot_mosaic(mosaico, figsize=(19.2, 12), layout='constrained')
    sns.barplot(data=plot_args[0], x=plot_args[1], y=plot_args[2], ax=axis['A'])

    axis['A'].xaxis.set_major_formatter('{x:,}')
    axis['A'].tick_params(axis='x', rotation=45)
    axis['A'].xaxis.tick_top()

    tabla_mas_altos = axis['B'].table(cellText=diez_mas_altos.values,
                                      colLabels=diez_mas_altos.columns,
                                      loc='center')
    tabla_mas_altos.scale(1, 2.3)
    tabla_mas_altos.auto_set_font_size(False)
    tabla_mas_altos.set_fontsize(10)


    tabla_mas_bajos = axis['C'].table(cellText=diez_mas_bajos.values,
                                      colLabels=diez_mas_bajos.columns,
                                      loc='center')
    
    tabla_mas_bajos.scale(1, 2.3)
    tabla_mas_bajos.auto_set_font_size(False)
    tabla_mas_bajos.set_fontsize(10)

    axis['B'].set_title('Top 10 gasto Neto')
    axis['C'].set_title('Bottom 10 gasto Neto')
    axis['B'].axis('off')
    axis['C'].axis('off')

    fig.suptitle(plot_args[3])
    plt.close()

    return fig


def analizar_distribucion_de_datos(df, columna_a_analizar, titulo_grafico):
    serie_columna = df[columna_a_analizar]
    descripcion_serie_columna = serie_columna.describe().to_frame().reset_index()
    descripcion_serie_columna = descripcion_serie_columna.rename(columns={'index': 'Estadisticas'})

    mosaico = '''
              AB
              AC
              '''

    fig, axis = plt.subplot_mosaic(mosaico, figsize=(19.2, 10.8), layout='constrained')
    args_plot = [df, columna_a_analizar, titulo_grafico]

    tabla = axis['A'].table(cellText=descripcion_serie_columna.values,
                            colLabels=descripcion_serie_columna.columns, loc='center')
    axis['A'].axis('off')

    sns.histplot(data=args_plot[0], x=args_plot[1], ax=axis['B'])
    sns.boxplot(data=args_plot[0], x=args_plot[1], ax=axis['C'])

    plt.ticklabel_format(style='plain', axis='x')

    axis['B'].tick_params(axis='x', rotation=45)
    axis['C'].tick_params(axis='x', rotation=45)
    axis['B'].xaxis.set_major_formatter('{x:,}')
    axis['C'].xaxis.set_major_formatter('{x:,}')

    fig.suptitle(args_plot[2])
    plt.close()

    return fig

def separar_por_cuartil(df_a_separar, columna_a_separar):
    qmin = df_a_separar[columna_a_separar].quantile(0)
    q1 = df_a_separar[columna_a_separar].quantile(0.25)
    q2 = df_a_separar[columna_a_separar].quantile(0.5)
    q3 = df_a_separar[columna_a_separar].quantile(0.75)
    qmax = df_a_separar[columna_a_separar].quantile(1)

    df_min_q1 = df_a_separar.query(
        f'`{columna_a_separar}` >= @qmin and `{columna_a_separar}` < @q1')
    df_q1_q2 = df_a_separar.query(
        f'`{columna_a_separar}` >= @q1 and `{columna_a_separar}` < @q2')
    df_q2_q3 = df_a_separar.query(
        f'`{columna_a_separar}` >= @q2 and `{columna_a_separar}` < @q3')
    df_q3_max = df_a_separar.query(
        f'`{columna_a_separar}` >= @q3 and `{columna_a_separar}` <= @qmax')

    return (df_min_q1, df_q1_q2, df_q2_q3, df_q3_max)

def analisis_global_y_cuartil(df_agrupada, eje_x_agrupado):
    imagenes_a_guardar = {}

    fig_rank_global = rankear_y_plottear(
        df_agrupada, 'Neto Total', eje_x_agrupado, 'Gasto por Servicio Global - Neto Total', 10)
    fig_distribucion_global = analizar_distribucion_de_datos(
        df_agrupada, 'Neto Total', 'Distribución Gasto Neto Total por Servicio Global')
    imagenes_a_guardar['Global'] = [fig_rank_global, fig_distribucion_global]

    cuartiles = separar_por_cuartil(df_agrupada, 'Neto Total')
    for i, cuartil in enumerate(cuartiles):
        if not cuartil.empty:
            fig_rank_cuartil = rankear_y_plottear(
                cuartil, 'Neto Total', eje_x_agrupado, f'Gasto por Servicio Intervalo Cuartil {i}', 10)
            fig_distribucion_cuartil = analizar_distribucion_de_datos(
                cuartil, 'Neto Total', f'Distribución Gasto Neto Intervalo Cuartil {i}')
            imagenes_a_guardar[i] = [fig_rank_cuartil, fig_distribucion_cuartil]
    
    return imagenes_a_guardar


def guardar_imagenes(imagenes_a_guardar, carpeta_a_guardar):
    for intervalo_imagen, lista_imagenes in imagenes_a_guardar.items():
        diccionario_intervalos = {'Global': 'Global', 0: 'INTERVALO_1_min_Q1',
                                  1: 'INTERVALO_2_Q1_Q2', 2: 'INTERVALO_3_Q2_Q3',
                                  3: 'INTERVALO_4_Q3_max'}
        path_nueva_carpeta = os.path.join(carpeta_a_guardar, diccionario_intervalos[intervalo_imagen])
        try:
            os.makedirs(path_nueva_carpeta)
        except FileExistsError:
            pass

        for i, imagen in enumerate(lista_imagenes):
            diccionario_nombres = {0: 'ranking', 1: 'distribucion'}
            tipo_archivo = diccionario_nombres[i]
            nombre_archivo = f'{diccionario_intervalos[intervalo_imagen]}_{tipo_archivo}.svg'
            ruta_archivo = os.path.join(path_nueva_carpeta, nombre_archivo)
            imagen.savefig(ruta_archivo)




In [3]:
df = pd.concat(map(lambda x: pd.read_csv(x, parse_dates=[
               0], dayfirst=True), obtener_full_path('input')))
df = df.sort_values('Fecha')
df_salidas = df.query('Movimiento == "Salida" and Cantidad > 0')


In [4]:
costos_por_unidades_global = df_salidas.groupby('Destino').sum().reset_index()
costos_por_unidades_global['Porcentaje Gasto'] = costos_por_unidades_global['Neto Total'] / \
                                                 costos_por_unidades_global['Neto Total'].sum()
costos_por_unidades_global = costos_por_unidades_global.drop(columns = ['Neto Unitario'])
costos_por_unidades_global = costos_por_unidades_global.round(2)
costos_por_unidades_global = costos_por_unidades_global.sort_values('Neto Total', ascending=False)
imagenes = analisis_global_y_cuartil(costos_por_unidades_global, eje_x_agrupado='Destino')

In [5]:
guardar_imagenes(imagenes, carpeta_a_guardar=os.path.join('output', 'gastos_globales_por_servicio'))

In [7]:
for i, servicio in enumerate(costos_por_unidades_global['Destino']):
    df_servicio = df_salidas.query('Destino == @servicio')
    agrupado_por_nombre = df_servicio.groupby('Nombre').sum().reset_index()
    agrupado_por_nombre['Porcentaje Gasto'] = agrupado_por_nombre['Neto Total'] / \
                                            agrupado_por_nombre['Neto Total'].sum()
    agrupado_por_nombre = agrupado_por_nombre.drop(columns = ['Neto Unitario'])
    agrupado_por_nombre = agrupado_por_nombre.round(2)
    agrupado_por_nombre = agrupado_por_nombre.sort_values('Neto Total', ascending=False)
    imagenes_servicio = analisis_global_y_cuartil(agrupado_por_nombre, eje_x_agrupado='Nombre')
    nombre_ranking_servicio = f'{i+1}_RANK_{servicio}'
    guardar_imagenes(imagenes_servicio, carpeta_a_guardar=os.path.join('output', 'articulos_por_servicio', nombre_ranking_servicio))


C:\Users\gmolina\AppData\Local\Temp\ipykernel_24708\17740028.py:138: UserWarning: Glyph 9 (	) missing from current font.
  imagen.savefig(ruta_archivo)
